<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liars in the Miller-Rabin Primality Test by Arjun Yadav
## All code is available on [GitHub](https://github.com/y-arjun-y/liars-miller-rabin).

## Implementation of the Miller-Rabin primality test in Python
### Credit goes to [Rosetta Code](https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python).

In [1]:
'''
Implementation of the Miller-Rabin primality test in Python. 
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python.
Variables and formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of False means n in not prime.
A return value of True means n is very likely to be a prime.
'''

def miller_rabin(n, a):
    # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    # assigning variables
    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials, directly proportional to time and accuracy.
    num_trials = 10

    # run
    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

## Length of probable primes function (using the Miller-Rabin primality test)

In [2]:
'''
Returns the length of prime numbers of a given range.
'''

def len_probable_prime(int, a):
    results = []

    # main loop
    for i in range(int + 1):
        if miller_rabin(i, a) == True:
            results.append(miller_rabin(i, a))

    return len(results)

## Actual number of primes between two numbers

In [9]:
'''
Returns the actual number of primes between two numbers.
Used in the final worst witness cell/file. Doesn't use the Miller-Rabin primality test.
Credit: https://www.programiz.com/python-programming/examples/prime-number-intervals
'''

def actual_len_primes(lower, upper):
  for num in range(lower, upper + 1):
    if num > 1:
        for i in range(2, num):
            if (num % i) == 0:
                break
        else:
            return num

## Finding worst witness in a certain range

In [12]:
# imports
from tqdm import tqdm

# constants
DEFENDANT = 91 # number we're using to test our function
WITNESS_RANGE = 91 # range till where we want to find the worst witness 
ACTUAL_LEN_PRIMES = actual_len_primes(0, WITNESS_RANGE)

# variables
results = {} # results before finding difference from the actual length of primes

# subtraction loop
for i in tqdm(range(WITNESS_RANGE)):
  results[i] = ACTUAL_LEN_PRIMES - len_probable_prime(DEFENDANT, i)

# finding worst key
print(list(results.keys())[list(results.values()).index(sorted(set(results.values()))[1])])

100%|██████████| 91/91 [00:00<00:00, 799.96it/s]

18


# Other functions created during research


## Length of composites function

In [17]:
'''
Returns the length of composite numbers of a given range.
'''

def len_composite(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == False:
            results.append(miller_rabin(i, a))

    return len(results)

## Composites function

In [15]:
'''
Returns the composite numbers of a given range.
'''

def composite(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == False:
            results.append(i)

    return results

## Primes function

In [14]:
'''
Returns the probable prime numbers of a given range.
'''

def probable_primes(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == True:
            results.append(i)

    return results

## Finding excellent, okay and false witnesses

In [ ]:
# imports
from tqdm import tqdm

# constants
DEFENDANT = 1000
WITNESS_RANGE = 100
ACTUAL_LEN_PRIMES = 168

# seconday variables 
excellent_witnesses = []  # = ACTUAL_LEN_PRIMES
okay_witnesses = []  # ±5 ACTUAL_LEN_PRIMES
false_witnessses = []  # != ACTUAL_LEN_PRIMES

# secondary loop
for i in tqdm(range(WITNESS_RANGE + 1)):
    if len_probable_prime(DEFENDANT, i) == ACTUAL_LEN_PRIMES:
        excellent_witnesses.append(i)
    elif len_probable_prime(DEFENDANT, i) - ACTUAL_LEN_PRIMES <= 5:
        okay_witnesses.append(i)
    elif len_probable_prime(DEFENDANT, i) != ACTUAL_LEN_PRIMES:
        false_witnessses.append(i)

100%|██████████| 101/101 [00:02<00:00, 36.43it/s]
